In [3]:
from data_loading import load_wumls_entries
import pandas as pd

In [2]:
entries = load_wumls_entries()

In [9]:
df = pd.DataFrame([entry.dict() for entry in entries])

In [11]:
df["source"].unique()

array(['WIKTIONARY', 'MSHGER', 'DMDICD10', 'MDRGER', 'WHOGER', 'ICPCGER',
       'DMDUMD', 'OMIM', 'SNOMEDCT_US', 'LNC-DE-CH', 'LNC-DE-DE', 'SPN',
       'LNC-DE-AT'], dtype=object)

In [12]:
df["source"].value_counts()

MDRGER         99061
MSHGER         80864
DMDICD10       11864
LNC-DE-DE      11059
LNC-DE-CH       4941
DMDUMD          3373
WHOGER          3332
WIKTIONARY      3082
ICPCGER          716
LNC-DE-AT        188
OMIM               1
SNOMEDCT_US        1
SPN                1
Name: source, dtype: int64